In [22]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from xgboost import plot_importance
from bayes_opt import BayesianOptimization

%matplotlib inline

In [3]:
import os
import torch
import torch.nn as nn

### 데이터를 불러오자

In [4]:
dir = os.getcwd()
train = pd.read_csv(dir + "/data/preprocessed_data/train_data.csv")
test = pd.read_csv(dir + "/data/preprocessed_data/test_data.csv")
valid = pd.read_csv(dir + "/data/preprocessed_data/valid_data.csv")

### 데이터 확인

In [5]:
train.head()

,cust_no,label,E1,E2,E3,E4,E5,E6,E10,E14,...,농축업,무직,사무원,생산직,서비스직,은퇴,전문직,정치인,판매원,프리랜서
0,0xb2d283b6,1.0,965,965.0,965.0,1,0,209.0,210.0,18169,...,0,0,0,0,0,0,0,0,0,1
1,0xb2d62fab,1.0,368,368.0,368.0,0,1,173.0,275.0,18260,...,0,0,0,0,0,0,0,0,1,0
2,0xb2d69cdb,1.0,199,199.0,199.0,0,1,41.0,6.0,18168,...,0,0,0,0,0,0,0,0,1,0
3,0xb2d942e8,-1.0,120,120.0,120.0,0,1,11.0,1.0,18261,...,0,0,0,0,0,0,0,0,1,0
4,0xb2d9156f,1.0,40,40.0,40.0,0,0,18.0,1.0,18169,...,0,0,0,0,0,0,0,0,1,0


In [9]:
train_fil = train.fillna(0.5)
test_fil = test.fillna(0.5)
valid_fil = valid.fillna(0.5)

### 데이터 label 변경

In [10]:
train_rep = train_fil
test_rep = test_fil
valid_rep = valid_fil
train_rep['label'].replace(-1,2,inplace=True)
test_rep['label'].replace(-1,2,inplace=True)
valid_rep['label'].replace(-1,2,inplace=True)

##### data split

In [11]:
x_train, y_train = train_rep.drop(['label','cust_no'], axis=1), train_rep["label"]
x_valid, y_valid = valid_rep.drop(['label','cust_no'],axis=1), valid_rep['label']
x_test, y_test = test_rep.drop(['label','cust_no'], axis=1), test_rep['label']

### 모델 최적화

In [ ]:
param_bounds = {'learning_rate':(0.1,0.3),
                'n_estimators':(20,100),
                'max_depth':(3,12),
                'min_child_weight' : (),
                'subample':(0.5,1),
                'gamma':(),
                'colsample_bytree':(),
                }

In [ ]:
def xgb_evaluate(max_depth, gamma, colsample_bytree, min_child_weight, eta, subsample):
    params = {'eval_metric': 'auc',
              'objective':'binary:logistic',
              'min_child_weight': min_child_weight,
              'max_depth': int(max_depth),
              'subsample': subsample,
              'eta': eta,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    model = xgb.XGBClassifier(params, )
    
    
    return cv_result['test-auc-mean'].iloc[-1]

In [25]:
model = xgb.XGBClassifier(eval_metric = 'auc')
model.fit(x_train, y_train)

In [23]:
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 73.02%
